In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import statsmodels.api as smf

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, silhouette_score, mean_squared_error, r2_score

from sklearn import datasets, linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn import svm
from sklearn.neural_network import MLPClassifier

In [37]:
eShop2022 = pd.read_csv('D:/Rahul/NEU-MPS_Analytics/6] Spring24/Capstone/Sponsor Data/2022_eShop.csv')
eShop2023 = pd.read_csv('D:/Rahul/NEU-MPS_Analytics/6] Spring24/Capstone/Sponsor Data/2023_eShop.csv')
eShop2024 = pd.read_csv('D:/Rahul/NEU-MPS_Analytics/6] Spring24/Capstone/Sponsor Data/2024_eShop.csv')

Non_eShop2022 = pd.read_csv('D:/Rahul/NEU-MPS_Analytics/6] Spring24/Capstone/Sponsor Data/2022_non-eShop.csv')
Non_eShop2023 = pd.read_csv('D:/Rahul/NEU-MPS_Analytics/6] Spring24/Capstone/Sponsor Data/2023_non-eShop.csv')
Non_eShop2024 = pd.read_csv('D:/Rahul/NEU-MPS_Analytics/6] Spring24/Capstone/Sponsor Data/2024_non-eShop.csv')

C:\Users\rsaga\AppData\Local\Temp\ipykernel_107260\1552782323.py:1: DtypeWarning:

Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.



In [47]:
# Concat the 'df1', 'df2' and 'df3' dataframes and create single dataframe named as 'df'
df = pd.concat([eShop2022,eShop2023,eShop2024,Non_eShop2022,Non_eShop2023,Non_eShop2024])
df.head()

,order_date,order_number,material_number,order_material_net_value,sales_channel,PRH1,PRH4,customer_group,customer_number,region,DSO_Ind
0,2022-05-12,112140438,034.053V4,40.30,eShop,Implant Solutions,"Instruments, Cases + Others",University,30303430353030353632,NaN,Non-DSO
1,2022-11-28,112656875,109.945,1559.51,eShop,Implant Solutions,"BLAT Ti Implants, standard",General dentist,30303430363032353236,NaN,DSO
2,2022-01-31,111858749,118.357,211.11,eShop,Restorative Solutions,Other Prosthetics,Laboratory,30303430353039373330,NaN,Non-DSO
3,2022-02-28,111935927,024.0008S,39.19,eShop,Implant Solutions,BL Healing Surgical,Periodontist,30303430353239323337,NaN,Non-DSO
4,2022-02-01,111861821,025.2205,58.21,eShop,Restorative Solutions,Other Prosthetics,General dentist,30303430363233343333,NaN,Non-DSO


# Data Cleaning and Pre Processing:

In [48]:
# Check for data types:
df.dtypes

order_date                   object
order_number                  int64
material_number              object
order_material_net_value    float64
sales_channel                object
PRH1                         object
PRH4                         object
customer_group               object
customer_number              object
region                       object
DSO_Ind                      object
dtype: object

In [49]:
# Correct Date Format:
df['order_date'] = pd.to_datetime(df['order_date'], format='%Y-%m-%d')
print(df['order_date'].dtype)

datetime64[ns]


In [50]:
# Detecting Missing Values
df.isnull().sum()

order_date                        0
order_number                      0
material_number                   0
order_material_net_value          0
sales_channel                     0
PRH1                              0
PRH4                            219
customer_group                21563
customer_number                   0
region                      1504522
DSO_Ind                           0
dtype: int64

In [51]:
# Drop Missing Values:
df = df.dropna()

# Dashboard

In [57]:
import dash
from dash import dcc, html
import plotly.express as px
import plotly.graph_objects as go

# Top 10 Product Categories in PRH1 by Sales Channel
top_prh1_categories = df['PRH1'].value_counts().nlargest(10).index
filtered_df_prh1 = df[df['PRH1'].isin(top_prh1_categories)]
grouped_data_prh1 = filtered_df_prh1.groupby(['sales_channel', 'PRH1']).size().reset_index(name='Frequency')
grouped_data_prh1 = grouped_data_prh1.pivot(index='PRH1', columns='sales_channel', values='Frequency').fillna(0)
grouped_data_prh1 = grouped_data_prh1.stack().reset_index(name='Frequency')

category_comparison_fig = px.bar(grouped_data_prh1, x='PRH1', y='Frequency', color='sales_channel', barmode='group',
                                 title='Comparison of Top 10 Product Categories in PRH1 by Sales Channel',
                                 labels={'PRH1': 'Product Category', 'Frequency': 'Count'},
                                 color_continuous_scale='Viridis')

# Top 10 Product Sub-Categories in PRH4 by Sales Channel
top_prh4_subcategories = df['PRH4'].value_counts().nlargest(10).index
filtered_df_prh4 = df[df['PRH4'].isin(top_prh4_subcategories)]
grouped_data_prh4 = filtered_df_prh4.groupby(['sales_channel', 'PRH4']).size().reset_index(name='Frequency')
grouped_data_prh4 = grouped_data_prh4.pivot(index='PRH4', columns='sales_channel', values='Frequency').fillna(0)
grouped_data_prh4 = grouped_data_prh4.stack().reset_index(name='Frequency')

subcategory_comparison_fig = px.bar(grouped_data_prh4, x='PRH4', y='Frequency', color='sales_channel', barmode='group',
                                    title='Comparison of Top 10 Product Sub-Categories in PRH4 by Sales Channel',
                                    labels={'PRH4': 'Product Sub-Category', 'Frequency': 'Count'},
                                    color_continuous_scale='Viridis')

# Customer Group to Product Category (PRH1) Flow
subset_df_prh1 = df[['customer_group', 'PRH1', 'order_material_net_value']]
grouped_df_prh1 = subset_df_prh1.groupby(['customer_group', 'PRH1']).sum().reset_index()
customer_groups_prh1 = grouped_df_prh1['customer_group'].unique().tolist()
product_categories_prh1 = grouped_df_prh1['PRH1'].unique().tolist()
labels_prh1 = customer_groups_prh1 + product_categories_prh1
source_indices_prh1 = [labels_prh1.index(group) for group in grouped_df_prh1['customer_group']]
target_indices_prh1 = [labels_prh1.index(category) for category in grouped_df_prh1['PRH1']]

customer_category_sankey_fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=labels_prh1,
        color="blue"
    ),
    link=dict(
        source=source_indices_prh1,
        target=target_indices_prh1,
        value=grouped_df_prh1['order_material_net_value']
    )
)])

customer_category_sankey_fig.update_layout(title_text="Customer Group to Product Category Flow", font_size=10)

# Customer Group to Product Sub-Category (PRH4) Flow
subset_df_prh4 = df[['customer_group', 'PRH4', 'order_material_net_value']]
grouped_df_prh4 = subset_df_prh4.groupby(['customer_group', 'PRH4']).sum().reset_index()
customer_groups_prh4 = grouped_df_prh4['customer_group'].unique().tolist()
product_categories_prh4 = grouped_df_prh4['PRH4'].unique().tolist()
labels_prh4 = customer_groups_prh4 + product_categories_prh4
source_indices_prh4 = [labels_prh4.index(group) for group in grouped_df_prh4['customer_group']]
target_indices_prh4 = [labels_prh4.index(category) for category in grouped_df_prh4['PRH4']]

customer_subcategory_sankey_fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=labels_prh4,
        color="blue"
    ),
    link=dict(
        source=source_indices_prh4,
        target=target_indices_prh4,
        value=grouped_df_prh4['order_material_net_value']
    )
)])

customer_subcategory_sankey_fig.update_layout(title_text="Customer Group to Product Sub-Category Flow", font_size=10)

# Bubble Chart of Sales by Region and Product Category
bubble_chart_data = df.groupby(['region', 'PRH1']).agg({'order_material_net_value': 'sum'}).reset_index()
bubble_chart_fig = px.scatter(bubble_chart_data, x='PRH1', y='region', size='order_material_net_value', color='region',
                              title='Sales by Region and Product Category',
                              labels={'PRH1': 'Product Category', 'region': 'Region', 'order_material_net_value': 'Sales'})

# Complex Pie Chart of Customer Groups
top_customer_groups = df['customer_group'].value_counts().nlargest(5)
other_groups = df['customer_group'].value_counts()[5:].sum()
top_customer_groups['Other'] = other_groups
pie_chart_fig = px.pie(values=top_customer_groups.values, names=top_customer_groups.index, title='Distribution of Top Customer Groups',
                       color_discrete_sequence=px.colors.sequential.Viridis)

app = dash.Dash(__name__)

app.layout = html.Div([
    html.Div([
        dcc.Graph(figure=category_comparison_fig, id='category-comparison-graph'),
        dcc.Graph(figure=subcategory_comparison_fig, id='subcategory-comparison-graph')
    ], style={'display': 'flex', 'width': '100%'}),
    html.Div([
        dcc.Graph(figure=customer_category_sankey_fig, id='customer-category-sankey'),
        dcc.Graph(figure=customer_subcategory_sankey_fig, id='customer-subcategory-sankey')
    ], style={'display': 'flex', 'width': '100%'}),
    html.Div([
        dcc.Graph(figure=bubble_chart_fig, id='bubble-chart-graph'),
        dcc.Graph(figure=pie_chart_fig, id='pie-chart-graph')
    ], style={'display': 'flex', 'width': '100%'})
])

if __name__ == '__main__':
    app.run_server(debug=True)
